In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from _plotly_future_ import v4_subplots
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import pickle 

In [2]:
# parameter settings
S_min   = 50          # lower bound for the asset price
S_max   = 150         # upper bound for the asset price
Tau_min = 0.01        # lower bound for time to maturity
Tau_max = 1           # upper bound for time to maturity
K       = 100         # strike price
r       = 0.1         # risk-free interest rate                  
sig     = 0.85        # volatility               
d       = 0.2         # dividend rate                
steps   = 60          # steps 

In [3]:
def calculate_veta(S,K,r,d,Tau,sig):
    d1 = (np.log(S/K) + (r-d+sig**2/2) * Tau) / (sig*np.sqrt(Tau))
    d2 = d1 - sig*np.sqrt(Tau)
    vega = np.exp(-d*Tau) * S * np.sqrt(Tau) * norm.pdf(d1)
    return vega * (d + d1*(r-d)/(sig*np.sqrt(Tau)) - (1+d1*d2)/(2*Tau))

In [4]:
# Veta surface
Veta = np.zeros([steps, steps])

# space partitioning
Tau_grid = np.arange(Tau_min, Tau_max+0.01, (Tau_max-Tau_min)/(steps-1))
S_grid = np.arange(S_min, S_max+0.01, (S_max-S_min)/(steps-1))

for i, S in enumerate(S_grid):
    for j, Tau in enumerate(Tau_grid):
        Veta[i][j] = calculate_veta(S, K, r, d, Tau, sig) 

In [5]:
layout = go.Layout(title='Veta as function of the time to maturity and the asset price', 
                   scene = dict(xaxis_title = 'Tau', yaxis_title = 'S', zaxis_title = 'Veta', 
                                camera_eye = dict(x=-1.6, y=-1.6, z=0.7)))

fig = go.Figure(data=[go.Surface(z=Veta, x=Tau_grid, y=S_grid)], layout=layout)

# interactive plot
iplot(fig)

plot(fig, filename='Veta.html') 

'Veta.html'